# 🚀 AI Generator Agent Tutorial (Advanced): Advanced Production Features

Welcome to **Part 3** of the AI Generator Agent tutorial! This builds on the concepts from Part 2A and demonstrates the advanced, production-ready features used in the Orion AI Agent System.

## 🎯 What You'll Learn

- **LangChain Integration**: Professional AI workflow orchestration
- **GPT Optimization**: Advanced model configuration and validation
- **Repository-Aware Context**: Intelligent codebase analysis
- **Code Modification**: Precise file editing with validation
- **State Management**: Persistent tracking and analytics
- **Multi-Agent Context Passing**: Integration with other AI agents

## 🔄 Prerequisites

- Completed Part 2 (Core Concepts Tutorial)
- OpenRouter API key (for GPT access)
- Understanding of LangChain concepts
- Familiarity with Pydantic models

## 🏗️ Architecture Overview

This tutorial demonstrates the production architecture used in Orion:

```
LangChain Orchestration
    ↓
GPT Model (Optimized)
    ↓
Repository Context Analysis
    ↓
Structured Output (Pydantic)
    ↓
File Operations (Create/Modify)
    ↓
State Management & Tracking
```

## 🔑 OpenRouter Configuration for Advanced Models

Let's set up OpenRouter with advanced model validation and optimization for code generation.

In [74]:
import os
import time
import json
from pydantic import BaseModel, Field
from typing import Any, Dict, List, Optional

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser

In [75]:
# Check for OpenRouter API key (same as Part 2A)
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

## 🏗️ Enhanced Pydantic Models for Production

Let's create the advanced Pydantic models used in the production system.

In [76]:
# Enhanced Pydantic model matching the production system
class CodeGenerationResponse(BaseModel):
    """Enhanced structured response for production code generation."""

    success: bool = Field(description="Whether the generation was successful")
    files: List[Dict[str, str]] = Field(
        description="Generated files with name and content"
    )
    modifications: List[Dict[str, str]] = Field(
        description="File modifications with target and changes", default=[]
    )
    reasoning: str = Field(
        description="Step-by-step explanation of the approach and decisions"
    )
    dependencies: List[str] = Field(description="Required dependencies", default=[])
    next_steps: List[str] = Field(description="Recommended next steps", default=[])

In [77]:
# Base Agent State Management (simplified version)
class BaseAgentState:
    """Simple state management for our advanced AI generator."""

    def __init__(self, agent_name: str):
        self.agent_name = agent_name
        self.state = {}
        self.debug = False

    def update_state(self, key: str, value: Any) -> None:
        """Update state value."""
        self.state[key] = value

    def get_state(self, key: str, default: Any = None) -> Any:
        """Get state value with default."""
        return self.state.get(key, default)

    def log(self, message: str, level: str = "info") -> None:
        """Simple logging function."""
        timestamp = time.strftime("%H:%M:%S")
        level_emoji = {"info": "ℹ️", "error": "❌", "warning": "⚠️", "success": "✅"}
        emoji = level_emoji.get(level, "ℹ️")
        print(f"[{timestamp}] {emoji} [{self.agent_name}] {message}")

    def execute_with_tracking(self, operation_name: str, operation_func):
        """Execute operation with tracking."""
        start_time = time.time()
        self.log(f"Starting operation: {operation_name}")

        result = operation_func()
        duration = time.time() - start_time
        self.log(
            f"Operation {operation_name} completed in {duration:.2f}s", "success"
        )
        return result

## 🧠 Advanced AI Generator with LangChain Integration

Now let's build the production-ready AI Generator with LangChain orchestration and GPT-5 optimization.

In [78]:
class AdvancedAIGeneratorAgent(BaseAgentState):
    """
    Production-ready AI Generator Agent with LangChain and OpenRouter.

    Features:
    - Advanced model support through OpenRouter
    - LangChain orchestration for complex workflows
    - Repository-aware context building
    - Code modification capabilities
    - Production error handling and fallbacks
    """

    def __init__(
        self,
        model: str = "openai/gpt-4o-mini",
        temperature: float = 0.1,
        debug: bool = False,
    ):
        """Initialize the Advanced AI Generator Agent."""
        super().__init__("AdvancedAIGenerator")
        self.model = model
        self.temperature = temperature
        self.debug = debug

        # Initialize LangChain components with OpenRouter (without JSON response format)
        self.llm = ChatOpenAI(
            model=model,
            temperature=temperature,
            max_tokens=32768,  # Advanced models support large outputs
            api_key=openrouter_api_key,
            base_url="https://openrouter.ai/api/v1",
        )

        self.log(f"🤖 Initialized LangChain with OpenRouter model: {model}")

        # Initialize state
        self.update_state("model", model)
        self.update_state("temperature", temperature)
        self.update_state("generated_code", [])
        self.update_state("created_files", [])
        self.update_state("modified_files", [])

    def build_repository_context(self, repo_path: str) -> str:
        """Build intelligent repository context by analyzing existing code."""
        if not os.path.exists(repo_path):
            return "Repository path does not exist"

        context_parts = []
        supported_extensions = (
            ".py",
            ".js",
            ".ts",
            ".java",
            ".go",
            ".rs",
            ".cpp",
            ".c",
        )

        try:
            files_found = []
            total_lines = 0

            # Walk through repository and analyze files
            for root, _, filenames in os.walk(repo_path):
                for filename in filenames[:15]:  # Limit to first 15 files for context
                    if filename.endswith(supported_extensions):
                        rel_path = os.path.relpath(
                            os.path.join(root, filename), repo_path
                        )
                        files_found.append(rel_path)

                        # Analyze file content for additional context
                        try:
                            full_path = os.path.join(root, filename)
                            with open(full_path, "r", encoding="utf-8") as f:
                                content = f.read()
                                total_lines += len(content.split("\n"))
                        except:
                            pass  # Skip files that can't be read

            if files_found:
                context_parts.append(
                    f"- Repository contains {len(files_found)} code files ({total_lines} total lines)"
                )
                context_parts.append(f"- Key files: {', '.join(files_found[:10])}")

                # Detect project type based on files
                if any(f.endswith(".py") for f in files_found):
                    context_parts.append("- Project type: Python")
                if any(f.endswith((".js", ".ts")) for f in files_found):
                    context_parts.append("- Project type: JavaScript/TypeScript")
                if any(f.endswith(".java") for f in files_found):
                    context_parts.append("- Project type: Java")

                # Check for common framework files
                framework_indicators = {
                    "requirements.txt": "Python project with pip dependencies",
                    "package.json": "Node.js project with npm dependencies",
                    "Cargo.toml": "Rust project with cargo dependencies",
                    "pom.xml": "Java project with Maven dependencies",
                    "build.gradle": "Java project with Gradle dependencies",
                }

                for indicator, description in framework_indicators.items():
                    if os.path.exists(os.path.join(repo_path, indicator)):
                        context_parts.append(f"- {description}")

        except Exception as e:
            context_parts.append(f"- Repository analysis failed: {str(e)}")

        return (
            "\n".join(context_parts)
            if context_parts
            else "No code files found in repository"
        )

    def build_enhanced_context(self, context: Optional[Dict], repo_path: str) -> str:
        """Build enhanced context combining user context and repository analysis."""
        context_parts = []

        # Add user-provided context
        if context:
            context_parts.append("USER CONTEXT:")
            for key, value in context.items():
                if isinstance(value, list):
                    value = ", ".join(str(v) for v in value)
                context_parts.append(f"- {key}: {value}")

        # Add repository context
        repo_context = self.build_repository_context(repo_path)
        if repo_context:
            context_parts.append("\nREPOSITORY CONTEXT:")
            context_parts.append(repo_context)

        return (
            "\n".join(context_parts)
            if context_parts
            else "No additional context provided"
        )


print("🏗️ Advanced AI Generator Agent class structure defined!")

🏗️ Advanced AI Generator Agent class structure defined!


In [79]:
def generate_code_changes(self, prompt: str, repo_path: str, context: Optional[Dict] = None) -> Optional[str]:
    """
    Generate code changes using advanced models through OpenRouter with LangChain orchestration.
    
    Args:
        prompt: Task description for the AI
        repo_path: Path to the repository
        context: Additional context for generation
    
    Returns:
        str: Generated code changes with reasoning, or None if failed
    """

    def _generation_operation():
        # Enhanced prompt template for advanced models with reasoning
        template = ChatPromptTemplate.from_messages([
            (
                "system",
                """You are an expert software architect using advanced AI capabilities.

CORE COMPETENCIES:
- Software architecture and design patterns
- Modern development practices and frameworks
- Code quality, security, and performance optimization
- Cross-platform and cloud-native development

OUTPUT REQUIREMENTS:
You MUST respond with valid JSON in this exact structure:
{{
  "success": true,
  "files": [
    {{"name": "filename.py", "content": "complete production-ready code"}},
    {{"name": "requirements.txt", "content": "package==version\\npackage2==version"}}
  ],
  "modifications": [
    {{"target": "existing_file.py", "changes": "specific changes to make"}}
  ],
  "reasoning": "Detailed step-by-step analysis of the problem, solution approach, design decisions, and trade-offs considered",
  "dependencies": ["package1", "package2"],
  "next_steps": ["step 1", "step 2"]
}}

REASONING PROCESS:
1. ANALYZE: Break down requirements and constraints
2. DESIGN: Plan architecture and component interactions
3. IMPLEMENT: Generate production-ready code with best practices
4. VALIDATE: Consider edge cases, security, and performance

QUALITY STANDARDS:
- Complete, runnable code with all imports
- Comprehensive error handling and logging
- Type hints and detailed Google-style docstrings
- Security best practices and input validation
- Performance optimization and resource management

Please provide your response in JSON format as specified above."""
            ),
            (
                "user",
                """DEVELOPMENT TASK:
Repository: {repo_path}
Task: {prompt}

CONTEXT:
{context}

REQUIREMENTS:
Generate a complete, production-ready solution that follows best practices.
Include detailed reasoning about your approach and design decisions.
Consider scalability, maintainability, and security in your solution.
Respond with valid JSON as specified in the system message."""
            )
        ])
        
        # Use string output parser and handle JSON manually
        chain = template | self.llm | StrOutputParser()
        
        # Build enhanced context
        context_str = self.build_enhanced_context(context, repo_path)
        
        try:
            response = chain.invoke({
                "prompt": prompt,
                "repo_path": repo_path,
                "context": context_str
            })
            
            # Try to parse JSON from response
            try:
                # Clean the response - remove markdown code blocks if present
                json_str = response.strip()
                if json_str.startswith('```json'):
                    json_str = json_str[7:]  # Remove ```json
                if json_str.startswith('```'):
                    json_str = json_str[3:]   # Remove ```
                if json_str.endswith('```'):
                    json_str = json_str[:-3]  # Remove closing ```
                    
                result = json.loads(json_str)
                
            except json.JSONDecodeError:
                self.log("❌ Failed to parse JSON response, using fallback", "warning")
                return self._fallback_generation(prompt, repo_path, context_str)
            
            # Format result for output
            if isinstance(result, dict):
                formatted_result = self._format_structured_result(result)
            else:
                formatted_result = str(result)
            
            # Update state with generation info
            generation_info = {
                "prompt": prompt,
                "repo_path": repo_path,
                "context": context,
                "result": result,
                "formatted_result": formatted_result,
                "timestamp": time.time(),
                "model": self.model,
                "success": result.get("success", False) if isinstance(result, dict) else True,
                "reasoning": result.get("reasoning", "") if isinstance(result, dict) else "",
            }
            
            generated_code = self.get_state("generated_code", [])
            generated_code.append(generation_info)
            self.update_state("generated_code", generated_code)
            
            return formatted_result
            
        except Exception as e:
            self.log(f"❌ Advanced model generation failed: {e}", "error")
            return self._fallback_generation(prompt, repo_path, context_str)
    
    return self.execute_with_tracking("generate_code_changes", _generation_operation)

AdvancedAIGeneratorAgent.generate_code_changes = generate_code_changes

In [80]:
def _format_structured_result(self, result: Dict) -> str:
    """Format structured result back to expected string format."""
    formatted_parts = []
    
    # Add reasoning as comment
    if result.get("reasoning"):
        reasoning_lines = result["reasoning"].split("\n")
        formatted_reasoning = "\n# ".join(["REASONING:"] + reasoning_lines)
        formatted_parts.append(f"# {formatted_reasoning}\n")
    
    # Format file creations
    for file_info in result.get("files", []):
        name = file_info.get("name", "unknown.py")
        content = file_info.get("content", "")
        formatted_parts.append(f"FILE: {name}\n```python\n{content}\n```\n")
    
    # Format modifications
    for mod_info in result.get("modifications", []):
        target = mod_info.get("target", "unknown.py")
        changes = mod_info.get("changes", "")
        formatted_parts.append(f"MODIFY: {target}\n```\n{changes}\n```\n")
    
    # Add dependencies and next steps as comments
    if result.get("dependencies"):
        deps = ", ".join(result["dependencies"])
        formatted_parts.append(f"# DEPENDENCIES: {deps}\n")
    
    if result.get("next_steps"):
        steps = "\n# ".join(result["next_steps"])
        formatted_parts.append(f"# NEXT STEPS:\n# {steps}\n")
    
    return "\n".join(formatted_parts)

AdvancedAIGeneratorAgent._format_structured_result = _format_structured_result

In [81]:
def _fallback_generation(self, prompt: str, repo_path: str, context_str: str) -> str:
    """Fallback generation method using simpler approach."""
    self.log("🔄 Using fallback generation method...", "warning")
    
    fallback_template = ChatPromptTemplate.from_messages([
        (
            "system",
            """You are an expert Python developer. Generate complete, production-ready code files.

OUTPUT FORMAT:
For each file, use this EXACT format:

FILE: filename.py
```python
# Complete file content with imports, classes, functions
# Include proper docstrings and type hints
```

REQUIREMENTS:
- Complete, runnable code
- All necessary imports
- Google-style docstrings
- Type hints
- Error handling
- Follow PEP 8"""
        ),
        (
            "user",
            "Repository: {repo_path}\nTask: {prompt}\nContext: {context}"
        )
    ])
    
    chain = fallback_template | self.llm | StrOutputParser()
    return chain.invoke({
        "prompt": prompt,
        "repo_path": repo_path,
        "context": context_str
    })

AdvancedAIGeneratorAgent._fallback_generation = _fallback_generation

In [82]:
# Verify that the change isn't too drastic
from difflib import SequenceMatcher

def modify_existing_file(
    self,
    repo_path: str,
    file_path: str,
    modification_description: str,
    file_content: Optional[str] = None,
) -> Optional[str]:
    """
    Modify an existing file with precision using advanced models.

    Args:
        repo_path: Repository path
        file_path: Path to file to modify
        modification_description: Description of changes to make
        file_content: Optional file content (will read from disk if not provided)

    Returns:
        str: Modified file content or None if failed
    """

    try:
        self.log(f"🔧 Modifying file with advanced model: {file_path}")

        # Construct full file path
        if os.path.isabs(file_path):
            full_file_path = file_path
        else:
            full_file_path = os.path.join(repo_path, file_path)

        # Read existing content
        if file_content is None:
            if not os.path.exists(full_file_path):
                self.log(f"❌ File not found: {full_file_path}", "error")
                return None

            with open(full_file_path, "r", encoding="utf-8") as f:
                existing_content = f.read()
        else:
            existing_content = file_content

        self.log(f"📄 Original file content ({len(existing_content)} chars)")

        # Use advanced model for precise modification with precision prompts
        modification_template = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """You are a precise code editor using advanced AI capabilities.

TASK: Modify code files with precision while preserving all existing functionality.

CRITICAL RULES:
- Make ONLY the requested change - do not refactor, reorganize, or "improve" other code
- Preserve ALL existing imports, comments, formatting, and structure
- Maintain the exact same file structure and organization
- Keep all existing functionality intact
- If changing a value, change ONLY that specific value
- If modifying a function, change ONLY what was requested

OUTPUT: Return the complete modified file content with only the minimal necessary change applied.""",
                ),
                (
                    "user",
                    """MODIFICATION REQUEST: {modification_description}

ORIGINAL FILE CONTENT:
```
{existing_content}
```

Apply the requested change with precision, keeping everything else exactly the same:""",
                ),
            ]
        )

        modification_chain = modification_template | self.llm | StrOutputParser()
        modified_content = modification_chain.invoke(
            {
                "existing_content": existing_content,
                "modification_description": modification_description,
            }
        )

        # Clean up the response (remove code block markers if present)
        modified_content = modified_content.strip()
        if modified_content.startswith("```"):
            lines = modified_content.split("\n")
            if len(lines) > 2:
                modified_content = "\n".join(lines[1:-1])

        # Verify that the change isn't too drastic using similarity check
        similarity = SequenceMatcher(None, existing_content, modified_content).ratio()

        if similarity < 0.5:  # If less than 50% similar, it's probably wrong
            self.log(
                f"❌ Modification too drastic (similarity: {similarity:.2f}), rejecting",
                "error",
            )
            return None

        self.log(
            f"✅ Advanced model modification applied (similarity: {similarity:.2f})"
        )

        # Write the modified content and track it
        self._write_modified_file(full_file_path, existing_content, modified_content)
        return modified_content

    except Exception as e:
        self.log(f"❌ Advanced model modification failed: {e}", "error")
        return None


def _write_modified_file(
    self, full_file_path: str, original_content: str, modified_content: str
) -> bool:
    """
    Write modified content to a file and track the modification.

    Args:
        full_file_path: Full path to the file
        original_content: Original file content
        modified_content: Modified file content

    Returns:
        bool: True if successful
    """
    try:
        with open(full_file_path, "w", encoding="utf-8") as f:
            f.write(modified_content)

        self.log(f"✅ Modified file: {full_file_path}")

        # Track modified file
        modified_files = self.get_state("modified_files", [])
        modified_files.append(
            {
                "path": full_file_path,
                "original_size": len(original_content),
                "modified_size": len(modified_content),
                "timestamp": time.time(),
                "similarity": SequenceMatcher(
                    None, original_content, modified_content
                ).ratio(),
            }
        )
        self.update_state("modified_files", modified_files)

        return True

    except Exception as e:
        self.log(f"❌ Failed to write modified file {full_file_path}: {e}", "error")
        return False


# Add modification methods to the class
AdvancedAIGeneratorAgent.modify_existing_file = modify_existing_file
AdvancedAIGeneratorAgent._write_modified_file = _write_modified_file

In [83]:
def get_model_info(self) -> Dict[str, Any]:
    """
    Get comprehensive information about the current advanced model.
    
    Returns:
        Dict: Model information and capabilities
    """
    return {
        "model": self.model,
        "temperature": self.temperature,
        "max_tokens": 32768,
        "supports_structured_output": True,
        "supports_reasoning": True,
        "context_window": 128000,  # Most advanced models support large context
        "provider": "OpenRouter",
        "langchain_integration": self.llm is not None,
        "state_management": True,
    }

def make_code_changes(self, generated_code: str, repo_path: str) -> bool:
    """
    Apply generated code changes to the repository.
    
    Args:
        generated_code: The generated code with FILE: and MODIFY: directives
        repo_path: Path to the repository
        
    Returns:
        bool: True if successful, False otherwise
    """
    try:
        self.log("📝 Applying generated code changes to repository")
        
        # Parse the generated code for FILE: and MODIFY: directives
        lines = generated_code.split('\n')
        current_file = None
        current_content = []
        files_created = []
        
        i = 0
        while i < len(lines):
            line = lines[i]
            
            # Check for FILE: directive
            if line.strip().startswith('FILE:'):
                # Save previous file if exists
                if current_file and current_content:
                    self._create_file(repo_path, current_file, '\n'.join(current_content))
                    files_created.append(current_file)
                
                # Extract filename
                current_file = line.replace('FILE:', '').strip()
                current_content = []
                
                # Skip to code block
                i += 1
                while i < len(lines) and not lines[i].strip().startswith('```'):
                    i += 1
                
                # Collect content until closing ```
                i += 1
                while i < len(lines) and not lines[i].strip().startswith('```'):
                    current_content.append(lines[i])
                    i += 1
                    
            i += 1
        
        # Save last file if exists
        if current_file and current_content:
            self._create_file(repo_path, current_file, '\n'.join(current_content))
            files_created.append(current_file)
        
        # Update state
        created_files = self.get_state("created_files", [])
        created_files.extend(files_created)
        self.update_state("created_files", created_files)
        
        self.log(f"✅ Successfully created {len(files_created)} files")
        return True
        
    except Exception as e:
        self.log(f"❌ Failed to apply code changes: {e}", "error")
        return False

def _create_file(self, repo_path: str, filename: str, content: str) -> bool:
    """
    Create a file in the repository.
    
    Args:
        repo_path: Repository path
        filename: Name of the file to create
        content: File content
        
    Returns:
        bool: True if successful
    """
    try:
        file_path = os.path.join(repo_path, filename)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(content)
            
        self.log(f"✅ Created file: {filename}")
        return True
        
    except Exception as e:
        self.log(f"❌ Failed to create file {filename}: {e}", "error")
        return False

# Add advanced file operation methods to the class
AdvancedAIGeneratorAgent.get_model_info = get_model_info
AdvancedAIGeneratorAgent.make_code_changes = make_code_changes
AdvancedAIGeneratorAgent._create_file = _create_file

In [84]:
# Create the advanced generator
advanced_generator = AdvancedAIGeneratorAgent(
    model="openai/gpt-4o-mini",  # Using GPT-4o-mini through OpenRouter
    temperature=0.1,  # Low temperature for deterministic code generation
    debug=True,  # Enable debug logging
)

print("🤖 Advanced AI Generator Agent created successfully!")

# Display model information
model_info = advanced_generator.get_model_info()
print("\n📊 Model Information:")
for key, value in model_info.items():
    print(f"   • {key}: {value}")

# Test repository context building
test_repo = "/Users/ishandutta/Documents/code/ai-accelerator/orion/ai_code_gen_demo"
print(f"\n📁 Creating test repository: {test_repo}")

# Create the test repository directory if it doesn't exist
os.makedirs(test_repo, exist_ok=True)

# Create some sample files for context testing
sample_files = {
    "main.py": "# Main application file\nif __name__ == '__main__':\n    print('Hello World')\n",
    "utils.py": "# Utility functions\ndef helper_function():\n    pass\n",
    "requirements.txt": "requests==2.28.0\nflask==2.2.0\n",
}

for filename, content in sample_files.items():
    file_path = os.path.join(test_repo, filename)
    with open(file_path, "w") as f:
        f.write(content)

print(f"✅ Created {len(sample_files)} sample files in test repository")

# Test repository context building
repo_context = advanced_generator.build_repository_context(test_repo)
print("\n🔍 Repository Context Analysis:")
print(repo_context)

[12:31:52] ℹ️ [AdvancedAIGenerator] 🤖 Initialized LangChain with OpenRouter model: openai/gpt-4o-mini
🤖 Advanced AI Generator Agent created successfully!

📊 Model Information:
   • model: openai/gpt-4o-mini
   • temperature: 0.1
   • max_tokens: 32768
   • supports_structured_output: True
   • supports_reasoning: True
   • context_window: 128000
   • provider: OpenRouter
   • langchain_integration: True
   • state_management: True

📁 Creating test repository: /Users/ishandutta/Documents/code/ai-accelerator/orion/ai_code_gen_demo
✅ Created 3 sample files in test repository

🔍 Repository Context Analysis:
- Repository contains 2 code files (8 total lines)
- Key files: utils.py, main.py
- Project type: Python
- Python project with pip dependencies


## 🎯 Advanced Code Generation Demo

Let's demonstrate the advanced code generation capabilities with repository-aware context.

In [85]:
# Advanced code generation task with rich context (Diffusers example)
advanced_prompt = (
    "Create a FastAPI server that exposes an endpoint to generate images using a Stable Diffusion model from Hugging Face Diffusers. The server should accept a text prompt and return the generated image."
)

# Provide comprehensive context
advanced_context = {
    "framework": "FastAPI",
    "diffusers_model": "runwayml/stable-diffusion-v1-5",
    "features": [
        "text-to-image endpoint",
        "asynchronous image generation",
        "input validation",
        "return image as base64 or downloadable file",
        "logging of requests",
    ],
    "requirements": [
        "production-ready code",
        "comprehensive error handling",
        "security best practices (e.g., input sanitization, rate limiting)",
        "proper project structure",
        "GPU support if available",
    ],
    "dependencies": [
        "diffusers",
        "torch",
        "transformers",
        "fastapi",
        "uvicorn",
        "pillow",
    ],
}

print(f"🎯 Advanced Generation Task: {advanced_prompt}")
print(f"📋 Context: {json.dumps(advanced_context, indent=2)}")
print("⏳ Generating with GPT  and repository awareness...")

# Generate code with advanced features
result = advanced_generator.generate_code_changes(
    prompt=advanced_prompt, repo_path=test_repo, context=advanced_context
)

if result:
    print("\n✅ Advanced code generation successful!")

    # Show generation statistics
    generation_history = advanced_generator.get_state("generated_code", [])
    if generation_history:
        latest = generation_history[-1]
        print(f"✅ Success: {latest.get('success', False)}")
        print(f"📄 Result length: {len(result)} characters")

    # Apply the generated code to the repository
    print("\n📝 Applying generated code to repository...")
    success = advanced_generator.make_code_changes(result, test_repo)

    if success:
        print("✅ Code changes applied successfully!")

        # Show created files
        created_files = advanced_generator.get_state("created_files", [])
        print(f"\n📁 Files created: {len(created_files)}")
        for file in created_files:
            print(f"   • {file}")

        # Show repository structure
        print("\n📂 Updated repository structure:")
        for root, dirs, files in os.walk(test_repo):
            level = root.replace(test_repo, "").count(os.sep)
            indent = "  " * level
            folder_name = os.path.basename(root) or "root"
            print(f"{indent}{folder_name}/")

            subindent = "  " * (level + 1)
            for file in files:
                file_path = os.path.join(root, file)
                size = os.path.getsize(file_path)
                print(f"{subindent}{file} ({size} bytes)")

🎯 Advanced Generation Task: Create a FastAPI server that exposes an endpoint to generate images using a Stable Diffusion model from Hugging Face Diffusers. The server should accept a text prompt and return the generated image.
📋 Context: {
  "framework": "FastAPI",
  "diffusers_model": "runwayml/stable-diffusion-v1-5",
  "features": [
    "text-to-image endpoint",
    "asynchronous image generation",
    "input validation",
    "return image as base64 or downloadable file",
    "logging of requests"
  ],
  "requirements": [
    "production-ready code",
    "comprehensive error handling",
    "security best practices (e.g., input sanitization, rate limiting)",
    "proper project structure",
    "GPU support if available"
  ],
  "dependencies": [
    "diffusers",
    "torch",
    "transformers",
    "fastapi",
    "uvicorn",
    "pillow"
  ]
}
⏳ Generating with GPT  and repository awareness...
[12:33:15] ℹ️ [AdvancedAIGenerator] Starting operation: generate_code_changes
[12:33:34] ✅ [Ad

## 🔧 Modification Demo

Let's demonstrate the code modification capabilities.

In [87]:
if advanced_generator and test_repo:
    # Create a sample file to modify
    sample_code = '''#!/usr/bin/env python3
"""
Sample application for testing modifications.
"""

import os
import sys

class Calculator:
    """A simple calculator class."""
    
    def __init__(self):
        self.version = "1.0.0"
    
    def add(self, a: float, b: float) -> float:
        """Add two numbers."""
        return a + b
    
    def subtract(self, a: float, b: float) -> float:
        """Subtract two numbers."""
        return a - b

if __name__ == "__main__":
    calc = Calculator()
    print(f"Calculator v{calc.version}")
    print(f"2 + 3 = {calc.add(2, 3)}")
'''

    # Write the sample file
    sample_file_path = os.path.join(test_repo, "calculator_sample.py")
    with open(sample_file_path, "w") as f:
        f.write(sample_code)

    print("🔧 Modification Demo")
    print(f"📁 Created sample file: calculator_sample.py")
    print(f"📄 Original file size: {len(sample_code)} characters")

    # Test modification
    modification_request = (
        "Update the version number from 1.0.0 to 2.1.0 and add a multiply method"
    )

    print(f"\n🎯 Modification request: {modification_request}")
    print("⏳ Applying modification...")

    modified_content = advanced_generator.modify_existing_file(
        repo_path=test_repo,
        file_path="calculator_sample.py",
        modification_description=modification_request,
    )

    if modified_content:
        print("\n✅ Modification successful!")
        print(f"📄 Modified file size: {len(modified_content)} characters")

        # Show modification statistics
        modified_files = advanced_generator.get_state("modified_files", [])
        if modified_files:
            latest_mod = modified_files[-1]
            print(f"🔄 Similarity score: {latest_mod.get('similarity', 0):.2%}")
            print(
                f"📊 Size change: {latest_mod.get('original_size', 0)} → {latest_mod.get('modified_size', 0)} chars"
            )

        # Show the differences (first 500 characters of each)
        print("\n📝 Content comparison (preview):")
        print("ORIGINAL (first 300 chars):")
        print(sample_code[:300] + "...")
        print("\nMODIFIED (first 300 chars):")
        print(modified_content[:300] + "...")

    else:
        print("❌ Modification failed!")
else:
    print("⚠️ Modification demo skipped (requires OpenAI API key)")

🔧 Modification Demo
📁 Created sample file: calculator_sample.py
📄 Original file size: 555 characters

🎯 Modification request: Update the version number from 1.0.0 to 2.1.0 and add a multiply method
⏳ Applying modification...
[12:35:54] ℹ️ [AdvancedAIGenerator] 🔧 Modifying file with advanced model: calculator_sample.py
[12:35:54] ℹ️ [AdvancedAIGenerator] 📄 Original file content (555 chars)
[12:36:01] ℹ️ [AdvancedAIGenerator] ✅ Advanced model modification applied (similarity: 0.90)
[12:36:01] ℹ️ [AdvancedAIGenerator] ✅ Modified file: /Users/ishandutta/Documents/code/ai-accelerator/orion/ai_code_gen_demo/calculator_sample.py

✅ Modification successful!
📄 Modified file size: 665 characters
🔄 Similarity score: 90.49%
📊 Size change: 555 → 665 chars

📝 Content comparison (preview):
ORIGINAL (first 300 chars):
#!/usr/bin/env python3
"""
Sample application for testing modifications.
"""

import os
import sys

class Calculator:
    """A simple calculator class."""

    def __init__(self):
      

## 📊 Advanced Analytics and State Management

Let's explore the comprehensive state management and analytics capabilities.

In [88]:
if advanced_generator:
    print("📊 Advanced Analytics Dashboard")
    print("=" * 50)

    # Model information
    model_info = advanced_generator.get_model_info()
    print(f"🤖 Model: {model_info['model']}")
    print(f"🌡️ Temperature: {model_info['temperature']}")
    print(f"📏 Context Window: {model_info['context_window']:,} tokens")
    print(f"🔗 LangChain: {model_info['langchain_integration']}")

    # Generation history
    generation_history = advanced_generator.get_state("generated_code", [])
    print(f"\n📈 Generation History: {len(generation_history)} generations")

    if generation_history:
        for i, gen in enumerate(generation_history, 1):
            print(f"\n{i}. 🎯 Generation {i}")
            print(f"   📝 Prompt: {gen.get('prompt', 'Unknown')[:50]}...")
            print(f"   ✅ Success: {gen.get('success', False)}")
            print(f"   ⏰ Time: {time.ctime(gen.get('timestamp', 0))}")

    # File operations
    created_files = advanced_generator.get_state("created_files", [])
    modified_files = advanced_generator.get_state("modified_files", [])

    print(f"\n📁 File Operations Summary:")
    print(f"   📄 Files created: {len(created_files)}")
    print(f"   🔧 Files modified: {len(modified_files)}")

    if created_files:
        print(f"\n📝 Created Files:")
        for file in created_files:
            print(f"   • {file}")

    if modified_files:
        print(f"\n🔧 Modified Files:")
        for mod in modified_files:
            similarity = mod.get("similarity", 0)
            timestamp = time.ctime(mod.get("timestamp", 0))
            print(
                f"   • {os.path.basename(mod.get('path', 'unknown'))} (similarity: {similarity:.1%}, {timestamp})"
            )

    # State overview
    print(f"\n🔍 State Overview:")
    for key, value in advanced_generator.state.items():
        if isinstance(value, list):
            print(f"   • {key}: {len(value)} items")
        else:
            print(f"   • {key}: {value}")

    print("\n🌟 Advanced Features Demonstrated:")
    print("   ✅ GPT-4o-mini integration with validation")
    print("   ✅ LangChain orchestration")
    print("   ✅ Repository-aware context building")
    print("   ✅ Code modification")
    print("   ✅ Advanced state management")
    print("   ✅ Comprehensive analytics")
    print("   ✅ Production error handling")

else:
    print("⚠️ Analytics dashboard not available (requires OpenAI API key)")

📊 Advanced Analytics Dashboard
🤖 Model: openai/gpt-4o-mini
🌡️ Temperature: 0.1
📏 Context Window: 128,000 tokens
🔗 LangChain: True

📈 Generation History: 1 generations

1. 🎯 Generation 1
   📝 Prompt: Create a FastAPI server that exposes an endpoint t...
   ✅ Success: True
   ⏰ Time: Sat Sep 27 12:33:34 2025

📁 File Operations Summary:
   📄 Files created: 2
   🔧 Files modified: 2

📝 Created Files:
   • main.py
   • requirements.txt

🔧 Modified Files:
   • calculator_sample.py (similarity: 90.5%, Sat Sep 27 12:35:34 2025)
   • calculator_sample.py (similarity: 90.5%, Sat Sep 27 12:36:01 2025)

🔍 State Overview:
   • model: openai/gpt-4o-mini
   • temperature: 0.1
   • generated_code: 1 items
   • created_files: 2 items
   • modified_files: 2 items

🌟 Advanced Features Demonstrated:
   ✅ GPT-4o-mini integration with validation
   ✅ LangChain orchestration
   ✅ Repository-aware context building
   ✅ Code modification
   ✅ Advanced state management
   ✅ Comprehensive analytics
   ✅ Productio

## 🎓 Advanced Tutorial Conclusion

Congratulations! You've completed **Part 2B: Advanced Production Features** of the AI Generator Agent tutorial. Here's what you've mastered:

### ✅ Production Architecture Mastered

1. **🔗 LangChain Integration**: Professional AI workflow orchestration with chains and parsers
2. **🤖 GPT-5 Optimization**: Advanced model configuration, validation, and feature detection
3. **🧠 Repository Intelligence**: Automated codebase analysis and context building
4. **🔧 Modifications**: Precise file editing with validation and safety checks
5. **🛡️ Production Resilience**: Fallback systems, error handling, and state management

### 🛠️ Advanced Technical Skills

- **LangChain Orchestration**: Building complex AI workflows with templates and chains
- **GPT-5 Integration**: Leveraging advanced model capabilities and structured outputs
- **Repository Analysis**: Intelligent codebase scanning and context extraction
- **State Management**: Persistent tracking of operations and analytics
- **Similarity Validation**: Safe code modification with change verification
- **Production Logging**: Comprehensive tracking and monitoring systems

### 🎯 Production Best Practices

- **Structured Output Validation**: Use Pydantic models for reliable AI responses
- **Context-Aware Generation**: Leverage repository analysis for better results
- **Precision**: Make minimal, targeted changes to preserve functionality
- **Fallback Systems**: Implement graceful degradation for resilience
- **State Persistence**: Track operations for analytics and debugging
- **Security Consciousness**: Follow best practices in generated code

### 🚀 Ready for Enterprise

You're now equipped to:
- Build production-ready AI code generation systems
- Integrate with existing development workflows
- Implement multi-agent AI systems like Orion
- Develop domain-specific AI agents
- Create enterprise-grade AI development tools

### 🔄 Connection to Orion System

This tutorial demonstrates the core architecture used in the Orion AI Agent System:
- **BaseAgent Integration**: State management and execution tracking
- **Multi-Agent Context**: Repository scanner and task classifier integration
- **LangGraph Orchestration**: Complex workflow coordination (covered in Tutorial 3)
- **Production Operations**: File management, testing, and deployment

---

*Continue to Tutorial 3 (LangGraph Orchestration) to learn how multiple AI agents work together in complex workflows!*

### 📚 Tutorial Series Navigation

- ✅ **Part 1**: GitHub Integration & Automation
- ✅ **Part 2A**: AI Generator Core Concepts  
- ✅ **Part 2B**: AI Generator Advanced Features (This Tutorial)
- 🔄 **Part 3**: LangGraph Orchestration (Next)
- 🔄 **Part 4**: Repository Scanner & Task Classifier
- 🔄 **Part 5**: Environment Management & Testing
- 🔄 **Part 6**: Complete System Integration